# Procesamiento de un interferograma con Stack Sentinel-1 TOPS


Los detalles del algoritmo para procesas stacks de datos tipo TOPS se puede encontrar en:

> Fattahi, H., P. Agram, and M. Simons (2016), A Network-Based Enhanced Spectral Diversity Approach for TOPS Time-Series Analysis, IEEE Transactions on Geoscience and Remote Sensing, 55(2), 777-786, doi:[10.1109/TGRS.2016.2614925](https://ieeexplore.ieee.org/abstract/document/7637021).

## Instalación

### 1. Instale ISCE2 
La instalación de ISCE2 se debe havcer de la manera usual (Ver los detalles [aquí](../README.md)). 
### 2. Paths de configuración para procesadores stack
Los procesadores de stack no aparecen en el directorio de instalación de su software ISCE. Se pueden encontrar en el directorio fuente de ISCE. Por lo tanto, se necesita una configuración de ruta adicional.
#### 2.1 Agregue el siguiente path 
Agregue el siguiente path a su variable de entorno ${PYTHONPATH}:
```
export ISCE_STACK={ruta_completa_a_tu_contrib/stack}
export PYTHONPATH=${PYTHONPATH}:${ISCE_STACK}
export PATH=${PATH}:${ISCE_STACK}/topsStack
```

## stackSentinel.py

Los scripts brindan soporte para el procesamiento de stack de TOPS de Sentinel-1. Los flujos de trabajo admitidos actualmente incluyen una pila registrada de SLC, interferogramas, compensaciones y coherencia.

`stackSentinel.py` genera todos los archivos de configuración y ejecución necesarios para ejecutarse en un stack de datos TOPS de Sentinel-1. Cuando se ejecuta stackSentinel.py para un flujo de trabajo determinado (opción -W), se genera una carpeta *configs* y *run_files*. En esta etapa no se realiza ningún procesamiento. Dentro de la carpeta *run_files* se encuentran diferentes archivos *run_#_descripción* que deben ejecutarse como scripts de shell en el orden del número de ejecución. Cada uno de estos scripts de ejecución llama a archivos de configuración específicos contenidos en la carpeta *configs* que llaman a ISCE de forma modular. Los archivos de configuración y ejecución cambiarán según el flujo de trabajo seleccionado. Para hacer que los archivos *run_#* sean ejecutables, cambie el permiso del archivo en consecuencia (por ejemplo, `chmod +x run_01_unpack_slc`).

Para ver las diferentes opciones de `stackSentinel.py`, podemos utilizar los siguiente comandos
```bash
stackSentinel.py -H #Para ver ejemplos de flujo de trabajo,
stackSentinel.py -h #Para obtener una descripción general de todos los parámetros configurables
```


In [3]:
!stackSentinel.py -H

This is the Open Source version of ISCE.
Some of the workflows depend on a separate licensed package.
To obtain the licensed package, please make a request for ISCE
through the website: https://download.jpl.nasa.gov/ops/request/index.cfm.
Alternatively, if you are a member, or can become a member of WinSAR
you may be able to obtain access to a version of the licensed sofware at
https://winsar.unavco.org/software/isce


Stack processor for Sentinel-1 data using ISCE software.

For a full list of different options, try stackSentinel.py -h

stackSentinel.py generates all configuration and run files required to be executed for a stack of Sentinel-1 TOPS data.

Following are required to start processing:

1) a folder that includes Sentinel-1 SLCs,
2) a DEM (Digital Elevation Model)
3) a folder that includes precise orbits (use dloadOrbits.py to download/ update your orbit folder. Missing orbits downloaded on the fly.)
4) a folder for Sentinel-1 Aux files (which is used for correcting the El

In [5]:
!stackSentinel.py -h

This is the Open Source version of ISCE.
Some of the workflows depend on a separate licensed package.
To obtain the licensed package, please make a request for ISCE
through the website: https://download.jpl.nasa.gov/ops/request/index.cfm.
Alternatively, if you are a member, or can become a member of WinSAR
you may be able to obtain access to a version of the licensed sofware at
https://winsar.unavco.org/software/isce
usage: stackSentinel.py [-h] [-H] -s SLC_DIRNAME -o ORBIT_DIRNAME -a
                        AUX_DIRNAME [-w WORK_DIR] -d DEM [-m REFERENCE_DATE]
                        [-c NUM_CONNECTIONS] [-n SWATH_NUM] [-b BBOX]
                        [-x EXCLUDE_DATES] [-i INCLUDE_DATES]
                        [--start_date STARTDATE] [--stop_date STOPDATE]
                        [-z AZIMUTHLOOKS] [-r RANGELOOKS] [-f FILTSTRENGTH]
                        [--snr_misreg_threshold SNRTHRESHOLD]
                        [-p POLARIZATION] [-C {geometry,NESD}]
                        [-O 

Los parámetros requeridos de `stackSentinel.py` incluyen:
```cfg
-s SLC_DIRNAME   #Una carpeta con Sentinel-1 SLC descargados.
-o ORBIT_DIRNAME #Una carpeta que contiene las órbitas de Sentinel-1. Los archivos de órbita faltantes se descargarán automáticamente
-a AUX_DIRNAME   #Una carpeta que contiene los archivos auxiliares de Sentinel-1
-d DEM_FILENAME  #Un DEM (Modelo de elevación digital) con referencia a wgs84
```

## 0.- Configuración inicial

De la misma forma que en el caso de topsApp.py, utilizaremos la siguiente estructura de directorios

```
.
├── Interferograma_StackSentinel.ipynb    (Este cuaderno)
├── stackproc                             (Aquí es donde procesaremos el interferograma)
└── data                                 (Aquí es donde descargaremos los datos para este portátil)
```

In [13]:
from shutil import copyfile, move # Utilidades para copiar y mover archivos
from osgeo import gdal            # Soporte GDAL para leer archivos virtuales
import os                         # Para crear y eliminar directorios
import matplotlib.pyplot as plt   # Para graficar
import numpy as np                # Cálculos de matrices
import glob                       # Recuperando lista de archivos
import boto3                      # Para hablar con el cubo s3

# directorio en el que reside el portátil
if 'tutorial_home_dir' not in globals():
    tutorial_home_dir = os.getcwd()
print("Directorio del notebook: ", tutorial_home_dir)
# directorio para descargas de datos
slc_dir = os.path.join(tutorial_home_dir,'data', 'slcs')
orbit_dir = os.path.join(tutorial_home_dir, 'data', 'orbits')

# definición de directorios de respaldo en caso de problemas de descarga en el servidor local
s3 = boto3.resource("s3")
data_backup_bucket = s3.Bucket("asf-jupyter-data")
data_backup_dir = "TOPS"

# generar todas las carpetas en caso de que aún no existan
os.makedirs(slc_dir, exist_ok=True)
os.makedirs(orbit_dir, exist_ok=True)
os.makedirs(insar_dir, exist_ok=True)

# Comience siempre en el directorio del notebook  
os.chdir(tutorial_home_dir)

# Utilidad para copiar datos de
def copy_from_bucket(file_in_bucket, dest_file,
                    bucket=data_backup_bucket):
    if os.path.exists(dest_file):
        print("Destination file {0} already exists. Skipping download...".format(dest_file))
    else:
        bucket.download_file(file_in_bucket, dest_file)

# Utilidad para graficar una matriz 2D
def plotdata(GDALfilename, band=1,
             title=None,colormap='gray',
             aspect=1, background=None,
             datamin=None, datamax=None,
             interpolation='nearest',
             nodata = None,
             draw_colorbar=True, colorbar_orientation="horizontal"):
    # Leer los datos en una matriz
    ds = gdal.Open(GDALfilename, gdal.GA_ReadOnly)
    data = ds.GetRasterBand(band).ReadAsArray()
    transform = ds.GetGeoTransform()
    ds = None
    try:
        if nodata is not None:
            data[data == nodata] = np.nan
    except:
        pass
    # obteniendo el min max de los ejes
    firstx = transform[0]
    firsty = transform[3]
    deltay = transform[5]
    deltax = transform[1]
    lastx = firstx+data.shape[1]*deltax
    lasty = firsty+data.shape[0]*deltay
    ymin = np.min([lasty,firsty])
    ymax = np.max([lasty,firsty])
    xmin = np.min([lastx,firstx])
    xmax = np.max([lastx,firstx])
    # poner todos los valores cero en nan y no trazar nan
    if background is None:
        try:
            data[data==0]=np.nan
        except:
            pass
    fig = plt.figure(figsize=(18, 16))
    ax = fig.add_subplot(111)
    cax = ax.imshow(data, vmin = datamin, vmax=datamax,
                    cmap=colormap, extent=[xmin,xmax,ymin,ymax],
                    interpolation=interpolation)
    ax.set_title(title)
    if draw_colorbar is not None:
        cbar = fig.colorbar(cax,orientation=colorbar_orientation)
    ax.set_aspect(aspect)    
    plt.show()
    # borrando los datos
    data = None

# Utilidad para trazar interferogramas
def plotcomplexdata(GDALfilename,
                    title=None, aspect=1,
                    datamin=None, datamax=None,
                    interpolation='nearest',
                    draw_colorbar=None, colorbar_orientation="horizontal"):
    # Cargue los datos en una matriz numpy
    ds = gdal.Open(GDALfilename, gdal.GA_ReadOnly)
    slc = ds.GetRasterBand(1).ReadAsArray()
    transform = ds.GetGeoTransform()
    ds = None
    # obteniendo el min max de los ejes
    firstx = transform[0]
    firsty = transform[3]
    deltay = transform[5]
    deltax = transform[1]
    lastx = firstx+slc.shape[1]*deltax
    lasty = firsty+slc.shape[0]*deltay
    ymin = np.min([lasty,firsty])
    ymax = np.max([lasty,firsty])
    xmin = np.min([lastx,firstx])
    xmax = np.max([lastx,firstx])
    # poner todos los valores cero en nan y no trazar nan
    try:
        slc[slc==0]=np.nan
    except:
        pass
    fig = plt.figure(figsize=(18, 16))
    ax = fig.add_subplot(1,2,1)
    cax1=ax.imshow(np.abs(slc), vmin = datamin, vmax=datamax,
                   cmap='gray', extent=[xmin,xmax,ymin,ymax],
                   interpolation=interpolation)
    ax.set_title(title + " (amplitude)")
    if draw_colorbar is not None:
        cbar1 = fig.colorbar(cax1,orientation=colorbar_orientation)
    ax.set_aspect(aspect)

    ax = fig.add_subplot(1,2,2)
    cax2 =ax.imshow(np.angle(slc), cmap='rainbow',
                    vmin=-np.pi, vmax=np.pi,
                    extent=[xmin,xmax,ymin,ymax],
                    interpolation=interpolation)
    ax.set_title(title + " (phase [rad])")
    if draw_colorbar is not None:
        cbar2 = fig.colorbar(cax2, orientation=colorbar_orientation)
    ax.set_aspect(aspect)
    plt.show()
    # borrando los datos
    slc = None

# Utilidad para trazar múltiples matrices similares
def plotstackdata(GDALfilename_wildcard, band=1,
                  title=None, colormap='gray',
                  aspect=1, datamin=None, datamax=None,
                  interpolation='nearest',
                  draw_colorbar=True, colorbar_orientation="horizontal"):
    # obtener una lista de todos los archivos que coincidan con los criterios de comodín de nombre de archivo
    GDALfilenames = glob.glob(GDALfilename_wildcard)
    # inicializar matriz numpy vacía
    data = None
    for GDALfilename in GDALfilenames:
        ds = gdal.Open(GDALfilename, gdal.GA_ReadOnly)
        data_temp = ds.GetRasterBand(band).ReadAsArray()   
        ds = None
        if data is None:
            data = data_temp
        else:
            data = np.vstack((data,data_temp))
    # poner todos los valores cero en nan y no trazar nan
    try:
        data[data==0]=np.nan
    except:
        pass            
    fig = plt.figure(figsize=(18, 16))
    ax = fig.add_subplot(111)
    cax = ax.imshow(data, vmin = datamin, vmax=datamax,
                    cmap=colormap, interpolation=interpolation)
    ax.set_title(title)
    if draw_colorbar is not None:
        cbar = fig.colorbar(cax,orientation=colorbar_orientation)
    ax.set_aspect(aspect)    
    plt.show() 
    # borrando los datos
    data = None

# Utilidad para trazar múltiples matrices complejas simples
def plotstackcomplexdata(GDALfilename_wildcard,
                         title=None, aspect=1,
                         datamin=None, datamax=None,
                         interpolation='nearest',
                         draw_colorbar=True, colorbar_orientation="horizontal"):
    # obtener una lista de todos los archivos que coincidan con los criterios de comodín de nombre de archivo
    GDALfilenames = glob.glob(GDALfilename_wildcard)
    print(GDALfilenames)
    # inicializar matriz numpy vacía
    data = None
    for GDALfilename in GDALfilenames:
        ds = gdal.Open(GDALfilename, gdal.GA_ReadOnly)
        data_temp = ds.GetRasterBand(1).ReadAsArray()
        ds = None
        if data is None:
            data = data_temp
        else:
            data = np.vstack((data,data_temp))
    # poner todos los valores cero en nan y no trazar nan
    try:
        data[data==0]=np.nan
    except:
        pass              
            
    fig = plt.figure(figsize=(18, 16))
    ax = fig.add_subplot(1,2,1)
    cax1=ax.imshow(np.abs(data), vmin=datamin, vmax=datamax,
                   cmap='gray', interpolation='nearest')
    ax.set_title(title + " (amplitude)")
    if draw_colorbar is not None:
        cbar1 = fig.colorbar(cax1,orientation=colorbar_orientation)
    ax.set_aspect(aspect)

    ax = fig.add_subplot(1,2,2)
    cax2 =ax.imshow(np.angle(data), cmap='rainbow',
                            interpolation='nearest')
    ax.set_title(title + " (phase [rad])")
    if draw_colorbar is not None:
        cbar2 = fig.colorbar(cax2,orientation=colorbar_orientation)
    ax.set_aspect(aspect)
    plt.show() 
    # borrando los datos
    data = None

Directorio del notebook:  /Users/javjrg/Library/CloudStorage/OneDrive-Personal/Proyectos/Gits/InSAR/Notebooks


## 0.1 Descarga de archivos SLC

In [16]:
# Actualice esto con su inicio de sesión de NASA Earthdata para descargar datos SLC
ASF_USER = "user"
ASF_PASS = "pass"

files = ['https://datapool.asf.alaska.edu/SLC/SA/S1A_IW_SLC__1SDV_20200511T135117_20200511T135144_032518_03C421_7768.zip',
         'https://datapool.asf.alaska.edu/SLC/SB/S1B_IW_SLC__1SDV_20200517T135026_20200517T135056_021622_0290CB_99E2.zip']
         
if len(ASF_USER)==0 or len(ASF_PASS)==0:
    raise Exception("Especifique su contraseña y usuario de ASF (inicio de sesión de Earthdata)")
    
for file in files:
    filename = os.path.basename(file)
    
    if not os.path.exists(os.path.join(slc_dir,filename)):
        cmd = "wget {0} --user={1} --password={2} -P {3} -nc".format(file, ASF_USER, ASF_PASS, slc_dir)
        print(cmd)
        os.system(cmd)
    else:
        print(filename + " ya existe. Saltando descarga...")

S1A_IW_SLC__1SDV_20200511T135117_20200511T135144_032518_03C421_7768.zip ya existe. Saltando descarga...
S1B_IW_SLC__1SDV_20200517T135026_20200517T135056_021622_0290CB_99E2.zip ya existe. Saltando descarga...
